<a href="https://colab.research.google.com/github/haricash/ms-project/blob/master/untrained_fsl_kdc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [ ]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 837 (delta 171), reused 181 (delta 130), pack-reused 539
Receiving objects: 100% (837/837), 2.22 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (460/460), done.
/content/easy-few-shot-learning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/easy-few-shot-learning
  Preparing metadata (setup.py) ... done
  Created wheel for easyfsl: filename=easyfsl-1.1.0-py3-none-any.whl size=45218 sha256=be87ba327f94b8296a3eebefa43a2ff1f05ad94557e1b563ebe4c9f5fc6a1f79
  Stored in directory: /root/.cache/pip/wheels/54/29/db/7eee3a758434d4337d8e4b1058b4b8cf2b33174f7f34da3ee9
Successfully built easyfsl


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

In [ ]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
n_way = 2
n_shot = 1
n_query = 1

DEVICE = "cuda"
n_workers = 12

In [ ]:
train_path = "/content/gdrive/MyDrive/KDC/train_loader.json"
valid_path = "/content/gdrive/MyDrive/KDC/valid_loader.json"

# Experimenting with no training and entire dataset for testing
test_path = "/content/gdrive/MyDrive/KDC/test_loader.json"
data_path = "/content/gdrive/MyDrive/KDC/data_loader.json"

In [ ]:
from easyfsl.datasets import EasySet
from torch.utils.data import DataLoader
from easyfsl.methods import PrototypicalNetworks
from easyfsl.samplers import TaskSampler
from easyfsl.modules import resnet12

In [ ]:
train_set = EasySet(train_path, 44)
val_set = EasySet(valid_path, 44)
data_set = EasySet(data_path, 44)

In [ ]:
model = resnet12(
    use_fc=True,
    num_classes=len(set(train_set.get_labels())),
).to(DEVICE)

In [ ]:
few_shot_classifier = PrototypicalNetworks(model).to(DEVICE)

In [ ]:
from easyfsl.methods.utils import evaluate

In [ ]:
n_test_tasks = 1000

# We'll load data path as we are experimenting without training on the entire dataset 
test_set = EasySet(data_path, 44)
test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [ ]:
model.set_use_fc(False)

accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 1000/1000 [00:19<00:00, 50.68it/s, accuracy=0.454]

Average accuracy : 45.35 %
